# Playground for downloading datasets

## Image net

In [78]:
import urllib
import requests
import json
import os
from glob import glob
import pandas as pd
import gbif_dl
from collections import Counter
import cv2
from PIL import Image

In [4]:
def store_raw_images():
    url = 'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n04154340'
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    urls = response.read().decode('utf-8')
    return(urls)

## Plantnet / GBIF

In [3]:
base_url = "https://api.gbif.org/v1/"
plantnet_key = "7a3679ef-5582-4aaa-81f0-8c2545cafc81"

PN_data_path = "../datasets/plantnet_raw"
PN_data_file = "verbatim.txt"

In [4]:
!ls {PN_data_path}

citations.txt  meta.xml      multimedia.txt  rights.txt
dataset        metadata.xml  occurrence.txt  verbatim.txt


In [5]:
fp = "../datasets/plantnet_raw/occurrence.txt"
nrow = 5
df = pd.read_csv(fp, nrows=nrow, sep='\t')
nas = df.isna().sum()
print(nas[nas < nrow].index)
df['speciesKey']

Index(['gbifID', 'identifier', 'license', 'publisher', 'references',
       'datasetName', 'basisOfRecord', 'occurrenceID', 'individualCount',
       'occurrenceStatus', 'eventDate', 'year', 'month', 'day', 'countryCode',
       'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters',
       'acceptedNameUsageID', 'scientificName', 'kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 'specificEpithet', 'taxonRank',
       'taxonomicStatus', 'datasetKey', 'publishingCountry', 'lastInterpreted',
       'issue', 'mediaType', 'hasCoordinate', 'hasGeospatialIssues',
       'taxonKey', 'acceptedTaxonKey', 'kingdomKey', 'phylumKey', 'classKey',
       'orderKey', 'familyKey', 'genusKey', 'speciesKey', 'species',
       'genericName', 'acceptedScientificName', 'verbatimScientificName',
       'protocol', 'lastParsed', 'lastCrawled', 'repatriated', 'level0Gid',
       'level0Name', 'level1Gid', 'level1Name', 'level2Gid', 'level2Name',
       'level3Gid', 'level3Name',

0    2927069
1    5361909
2    2890212
3    2927303
4    8351737
Name: speciesKey, dtype: int64

In [6]:
df = pd.read_csv(fp, sep='\t')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (198) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
test = df['countryCode'].value_counts()
test['US']

36256

In [55]:
test_2 = df[~df['speciesKey'].isna() &
          (df['countryCode'] == 'US')]['speciesKey'].astype('int').value_counts()
test_2

3039194    706
3084015    679
3120060    360
2926557    311
5376075    310
          ... 
7819589      1
3170627      1
8184083      1
5384458      1
5414964      1
Name: speciesKey, Length: 2488, dtype: int64

In [56]:
print(sum(test_2))

36250


In [24]:
(~df['speciesKey'].isna())

0         True
1         True
2         True
3         True
4         True
          ... 
794364    True
794365    True
794366    True
794367    True
794368    True
Name: speciesKey, Length: 794369, dtype: bool

In [111]:
species = [*df[~df['speciesKey'].isna() &
              (df['countryCode'] == 'US')]['speciesKey'].astype('int').value_counts().index]
print(len(species))
species[:5]

2488


[3039194, 3084015, 3120060, 2926557, 5376075]

In [129]:
# query = {
#     "speciesKey": [3039194, 2926986],
#     "datasetKey": [
#         "7a3679ef-5582-4aaa-81f0-8c2545cafc81",  # plantnet
#     ]
# }

query = {
    # "speciesKey": species[:200],
    "datasetKey": [
        "7a3679ef-5582-4aaa-81f0-8c2545cafc81",  # plantnet
    ]
}

In [130]:
data_generator = gbif_dl.api.generate_urls(
    queries=query,
    label="speciesKey",
    nb_samples=1000,
    nb_samples_per_stream=50
)

50


In [131]:
test = [*data_generator]

In [132]:
print(len(test))
test[:1]
print(Counter([val['label'] for val in test]))

50
Counter({'5356354': 2, '3145214': 2, '2891031': 2, '3190583': 2, '5373358': 1, '3190638': 1, '9419993': 1, '3094900': 1, '6375040': 1, '3062975': 1, '5295044': 1, '7334368': 1, '5330661': 1, '2769766': 1, '7307829': 1, '2984565': 1, '5334247': 1, '3172358': 1, '3189815': 1, '3034742': 1, '2868323': 1, '5384604': 1, '6035538': 1, '5330580': 1, '3025617': 1, '3172297': 1, '2684640': 1, '5415020': 1, '5371742': 1, '5367273': 1, '8971274': 1, '5415455': 1, '3152268': 1, '3021496': 1, '2858739': 1, '2766278': 1, '3171949': 1, '7262663': 1, '3033865': 1, '5334218': 1, '3172300': 1, '3117424': 1, '3085191': 1, '3066965': 1, '8848598': 1, '2889011': 1})


In [133]:
# stats = gbif_dl.io.download(data_generator, root="my_dataset")

In [2]:
test = requests.get(base_url + "dataset/" + plantnet_key + "/metrics")

NameError: name 'base_url' is not defined

In [83]:
for img_path in glob('my_dataset/**/*'):
    img = Image.open(img_path).convert('RGB')
    print(img.size)

(675, 900)
(675, 900)
(675, 900)
(425, 900)
(425, 900)
